In [1]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.9.0-py3-none-any.whl size=203107 sha256=c8916cdb2ad4fe196963dd70b783c85a1b6381c530d1bbb57f1bdefdade74c74
  Stored in directory: /root/.cache/pip/wheels/95/a7/fd/f667b71e29e3f64f239f30195efc39de642e72f8a47a6fcd8d
Successfully built pyTelegramBotAPI


In [2]:
import telebot
import torch
import numpy as np
import torch.nn as nn
import cv2
from PIL import Image
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.prep = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.PReLU()
        )
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.block_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.block_4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.block_5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )
        self.out_prep = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.BatchNorm2d(64),
        )
        self.out_conv_1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
        )
        self.out_p_relu_1 = nn.Sequential(
            nn.PReLU()
        )
        self.out_conv_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1),
        )
        self.out_p_relu_2 = nn.Sequential(
            nn.PReLU()
        )
        self.last_conv = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=3, kernel_size=9, stride=1, padding=4)
        )
    def forward(self, input):
        DEBUG = False
        if DEBUG:
            print('input.shape: ', input.shape)

        x = self.prep(input)
        if DEBUG:
            print('x.shape: ', x.shape)
        
        y1 = self.block_1(x)
        if(DEBUG):
            print('y1.shape: ', y1.shape)

        y1 = y1 + x
        y2 = self.block_2(y1)
        y2 = y2 + y1
        y3 = self.block_3(y2)
        y3 = y3 + y2
        y4 = self.block_4(y3)
        y4 = y4 + y3
        y5 = self.block_5(y4)
        y5 = y5 + y4

        if DEBUG:
            print('y5.shape: ', y5.shape, ' y4.shape: ', y4.shape)

        y5 = self.out_prep(y5)
        y5 = y5 + x

        if DEBUG:
            print('y5 out_prep.shape: ', y5.shape)

        pixel_shuffle = nn.PixelShuffle(2)

        y5 = self.out_conv_1(y5)

        if DEBUG:
            print('y5 out_conv_1.shape: ', y5.shape)

        y5 = pixel_shuffle(y5)
        if DEBUG:
            print('y5 pixel_shuffle1.shape: ', y5.shape)

        y5 = self.out_p_relu_1(y5)
        y5 = self.out_conv_2(y5)

        if DEBUG:
            print('y5 out_conv_2.shape: ', y5.shape)

        y5 = pixel_shuffle(y5)
        y5 = self.out_p_relu_2(y5)

        if DEBUG:
            print('y5 pixel_shuffle2.shape: ', y5.shape)

        y5 = self.last_conv(y5)

        if DEBUG:
            print('y5 last_conv.shape: ', y5.shape)

        return y5


In [4]:
generator = Generator().to(device)

In [6]:
generator.load_state_dict(torch.load('generator_v3', map_location=torch.device(device)))

<All keys matched successfully>

In [7]:
API_KEY_FILE_NAME = 'API_KEY.txt'
with open(API_KEY_FILE_NAME, "r") as file:
    API_KEY = file.readline()

In [32]:
bot = telebot.TeleBot(token=API_KEY)
@bot.message_handler(func=lambda x: True)
def handle_message(message):
    if message.content_type == 'text':
        print(message.text)

@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    raw = message.photo[2].file_id
    name = raw+".jpg"
    file_info = bot.get_file(raw)
    downloaded_file = bot.download_file(file_info.file_path)
    with open(name,'wb') as new_file:
        new_file.write(downloaded_file)
    
    photo = cv2.imread(name)[:,:,::-1]
    s = photo.shape
    photo = torch.from_numpy(photo.copy()).reshape(1, *s).permute(0, 3, 1, 2).type(torch.float32).to(device)
    generator.eval()
    with torch.no_grad():
        photo = generator(photo).cpu()
    s_new = photo.shape
    #print(s_new[3])
    photo = photo.reshape(s_new[1], s_new[2], s_new[3]).permute(1, 2, 0).numpy().astype(np.uint8)
    #print(photo)
    #plt.imshow(photo)
    #plt.show()
    pil_img = Image.fromarray(photo)
    
    pil_img.save('random_image.jpg')
    img = open('random_image.jpg', 'rb')
    bot.send_photo(message.chat.id, img)
    

In [ ]:
bot.polling()